In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from google.colab.data_table import DataTable
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, RegressorMixin
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import MinMaxScaler


ModuleNotFoundError: No module named 'google'

In [ ]:
filename=''
df = pd.read_csv(filename)
train, test=train_test_split(df,test_size=0.2,train_size=0.8,random_state=106)


In [ ]:
def split_and_create_columns(data):
  temp=pd.get_dummies(data.blood_type)
  data["A blood_type"]=temp['A+']+temp['A-']
  data["B or AB blood_type"]=temp['AB+']+temp['AB-']+temp['B-']+temp['B+']
  data["O blood_type"]=temp['O+']+temp['O-']
  temp=pd.get_dummies(data.symptoms)
  temp=temp.loc[:, ['smell_loss','cough;shortness_of_breath;sore_throat']]
  data = pd.concat([data, temp],axis=1)
  data['pcr_date'] = pd.to_datetime(data['pcr_date']).astype(int)
  data['sex']=np.where(data['sex']=='F',1,0)
  data[['Latitude','Longitude']] = data['current_location'].str.split(',', expand=True)
  data['Latitude']=data.Latitude.str.strip('(')
  data['Latitude']=data.Latitude.str.strip("'")
  data['Latitude']=data.Latitude.astype(float)
  data['Longitude']=data.Longitude.str.strip(')')
  data['Longitude']=data.Longitude.str.strip()
  data['Longitude']=data.Longitude.str.strip("'")
  data['Longitude'] = data['Longitude'].astype(float)
  return data


def prepare_data(training_data, new_data):
  training_data_c=training_data.copy(deep=True)
  new_data_c=new_data.copy(deep=True)
  training_data_c=split_and_create_columns(training_data_c)
  new_data_c=split_and_create_columns(new_data_c)
  for column in training_data_c:
    if(column=='age' or column=='Latitude' or column=='Longitude' or column=='num_of_siblings' or column=='happiness_score' or column=='household_income' or column=='sugar_levels' or column=='weight' or column=='PCR_04' or column=='PCR_05' or column=='PCR_06' or column=='PCR_10'):
      new_data_c[column]=(new_data_c[column]-training_data_c[column].mean())/training_data_c[column].std()
    if(column=='conversations_per_day' or column=='sport_activity' or column=='pcr_date' or column=='PCR_01' or column=='PCR_01' or column=='PCR_02' or column=='PCR_03' or column=='PCR_07' or column=='PCR_08' or column=='PCR_09'):
      new_data_c[column]=(new_data_c[column]-training_data_c[column].min())/(training_data_c[column].max()-training_data_c[column].min())
      new_data_c[column]=new_data_c[column]*2+(-1)
  return new_data_c[['age','sex','weight','A blood_type', 'B or AB blood_type', 'O blood_type', 'Latitude', 'Longitude', 'num_of_siblings',
 'happiness_score','household_income','conversations_per_day','sugar_levels','sport_activity','smell_loss',
 'cough;shortness_of_breath;sore_throat','pcr_date','PCR_01','PCR_02','PCR_03','PCR_04','PCR_05','PCR_06','PCR_07',
 'PCR_08','PCR_09','PCR_10','contamination_level']]

In [ ]:
test=prepare_data(train,test)
train=prepare_data(train,train)

In [ ]:
import pdb

class LinearRegressor(BaseEstimator, RegressorMixin):
    """
    Custom linear regression model
    """
    def __init__(self, lr: float = 1e-5):
        """
        Initialize an instance of this class.
        ** Do not edit this method **

        :param lr: the SGD learning rate (step size)
        """
        self.lr = lr
        self.batch_size = 32
        self.w = None
        self.b = 0.0

    # Initialize a random weight vector
    def init_solution(self, n_features: int):
        """
        Randomize an initial solution (weight vector)
        ** Do not edit this method **

        :param n_features:
        """
        self.w = np.zeros(n_features)
        self.b = 0.0

    @staticmethod
    def loss(w, b: float, X, y):
        """
        Compute the MSE objective loss.

        :param w: weight vector for linear regression; array of shape (n_features,)
        :param b: bias scalar for linear regression
        :param X: samples for loss computation; array of shape (n_samples, n_features)
        :param y: targets for loss computation; array of shape (n_samples,)
        :return: the linear regression objective loss (float scalar)
        """

        # TODO: complete the loss calculation
        loss = (np.linalg.norm(((X.dot(w)+b).reshape(-1)-y))**2)/(y.size)

        return loss

    @staticmethod
    def gradient(w, b: float, X, y):
        """
        Compute the (analytical) linear regression objective gradient.

        :param w: weight vector for linear regression; array of shape (n_features,)
        :param b: bias scalar for linear regression
        :param X: samples for loss computation; array of shape (n_samples, n_features)
        :param y: targets for loss computation; array of shape (n_samples,)
        :return: a tuple with (the gradient of the weights, the gradient of the bias)
        """
        # TODO: calculate the analytical gradient w.r.t w and b
        g_w = ((X.transpose()).dot(((X.dot(w)+b).reshape(-1)-y)))/(0.5*y.size)
        g_b = (((X.dot(w)+b).reshape(-1)-y).sum())/(0.5*y.size)

        return g_w, g_b

    def fit_with_logs(self, X, y, max_iter: int = 1000, keep_losses: bool = True,
                      X_val  =None, y_val = None):
        """
        Fit the model according to the given training data.

        :param X: training samples; array of shape (n_samples, n_features)
        :param y: training targets; array of shape (n_samples,)
        :param max_iter: number of SGD iterations
        :param keep_losses: should compute the train & val losses during training?
        :param X_val: validation samples to compute the loss for (for logs only)
        :param y_val: validation labels to compute the loss for (for logs only)
        :return: training and validation losses during training
        """
        # Initialize learned parameters
        self.init_solution(X.shape[1])

        train_losses = []
        val_losses = []

        if keep_losses:
            train_losses.append(self.loss(self.w, self.b, X, y))
            val_losses.append(self.loss(self.w, self.b, X_val, y_val))

        # Iterate over batches (SGD)
        for itr in range(0, max_iter):
            start_idx = (itr * self.batch_size) % X.shape[0]
            end_idx = min(X.shape[0], start_idx + self.batch_size)
            batch_X = X[start_idx: end_idx]
            batch_y = y[start_idx: end_idx]

            # TODO: Compute the gradient for the current *batch*
            g_w, g_b = self.gradient(self.w,self.b,batch_X,batch_y)

            # Perform a gradient step
            # TODO: update the learned parameters correctly
            self.w = self.w - self.lr*g_w
            self.b = self.b - self.lr*g_b

            if keep_losses:
                train_losses.append(self.loss(self.w, self.b,  X, y))
                val_losses.append(self.loss(self.w, self.b,  X_val, y_val))

        return train_losses, val_losses

    def fit(self, X, y, max_iter: int = 1000):
        """
        Fit the model according to the given training data.
        ** Do not edit this method **

        :param X: training samples; array of shape (n_samples, n_features)
        :param y: training targets; array of shape (n_samples,)
        :param max_iter: number of SGD iterations
        """
        self.fit_with_logs(X, y, max_iter=max_iter, keep_losses=False)

        return self

    def predict(self, X):
        """
        Regress labels on samples in X.

        :param X: samples for prediction; array of shape (n_samples, n_features)
        :return: Predicted continuous labels for samples in X; array of shape (n_samples,)
        """

        # TODO: Compute
        y_pred = (X.dot(self.w) + self.b).reshape(-1)

        return y_pred